# Retraining Model using Model Recombination (r2)

In [1]:
!pip install tensorflow[and-cuda] matplotlib numpy seaborn scikit-learn pillow

In [2]:
import tensorflow as tf
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import random
from tensorflow.keras.preprocessing import image

2024-09-04 21:28:43.077695: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-04 21:28:43.096039: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-04 21:28:43.101704: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-04 21:28:43.115490: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-04 21:28:44.049794: W tensorflow/compiler/tf2

## Load Your Existing Model

In [3]:
# Load your existing .keras model
your_model = load_model('./models/r1_expression_classification_model.keras')

2024-09-04 21:28:45.846017: I tensorflow/core/common_runtime/gpu/gpu_device.cc:2021] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18452 MB memory:  -> device: 0, name: NVIDIA RTX 4000 Ada Generation, pci bus id: 0000:c1:00.0, compute capability: 8.9


## Load a Public Model (e.g., MobileNetV2)

In [4]:
# Load a public model (MobileNetV2) without the top layers
public_model = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the layers of the public model
for layer in public_model.layers:
    layer.trainable = False

## Prepare Data for Training and Validation

In [5]:
# Prepare data
train_dir = './face-expression-recognition-dataset/images/train'
validation_dir = './face-expression-recognition-dataset/images/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 28821 images belonging to 7 classes.
Found 7066 images belonging to 7 classes.


## Combine Your Model with the Public Model

In [6]:
# Combine models
combined_model = public_model.output
combined_model = GlobalAveragePooling2D()(combined_model)

# Adjust your existing model to accept the combined model output
# Assuming 'your_model' is a sequential model, we'll replace its input layer
combined_model = Flatten()(combined_model)
combined_model = Dense(1024, activation='relu')(combined_model)  # Add a dense layer if needed

# Add final output layer based on the number of classes in your dataset
# Replace the last layer of your original model with one compatible with the number of classes
num_classes = train_generator.num_classes
output_layer = Dense(num_classes, activation='softmax')(combined_model)

# Define the final combined model
final_model = Model(inputs=public_model.input, outputs=output_layer)

## Compile and Train the Combined Model

### compile and get the model info

In [7]:
# Define the callbacks
checkpoint = ModelCheckpoint('./models/r2_best_model.keras', monitor='val_loss', save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001, mode='min')

# Combine all callbacks into a list
callbacks = [checkpoint, early_stopping, reduce_lr]

# Compile the model
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

final_model.summary()

Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 3,576,903 (13.64 MB)

 Trainable params: 1,318,919 (5.03 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

### model training

In [ ]:
# Train the model with callbacks
history = final_model.fit(
    train_generator,
    epochs=50,
    validation_data=validation_generator,
    callbacks=callbacks,
    shuffle=True
)

Epoch 1/50


/opt/micromamba/envs/python_310/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1725485339.349343   25196 service.cc:146] XLA service 0x7f99f003f1d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1725485339.349400   25196 service.cc:154]   StreamExecutor device (0): NVIDIA RTX 4000 Ada Generation, Compute Capability 8.9
2024-09-04 21:28:59.479461: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-04 21:29:00.056970: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 890

  1/901 ━━━━━━━━━━━━━━━━━━━━ 4:24:23 18s/step - accuracy: 0.1250 - loss: 2.3133

I0000 00:00:1725485349.004822   25196 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


162/901 ━━━━━━━━━━━━━━━━━━━━ 7:23 600ms/step - accuracy: 0.2810 - loss: 2.0146

2024-09-04 21:30:50.415252: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1395', 12 bytes spill stores, 12 bytes spill loads

2024-09-04 21:30:50.608191: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4280', 32 bytes spill stores, 32 bytes spill loads

2024-09-04 21:30:50.760449: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_4280', 32 bytes spill stores, 32 bytes spill loads

2024-09-04 21:30:50.796947: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1395', 196 bytes spill stores, 196 bytes spill loads

2024-09-04 21:30:50.901494: I external/local_xla/xla/strea

900/901 ━━━━━━━━━━━━━━━━━━━━ 0s 604ms/step - accuracy: 0.3334 - loss: 1.7394

2024-09-04 21:39:01.896141: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1184', 28 bytes spill stores, 28 bytes spill loads

2024-09-04 21:39:02.033190: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1184', 172 bytes spill stores, 172 bytes spill loads

2024-09-04 21:39:02.166307: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1191', 12 bytes spill stores, 12 bytes spill loads

2024-09-04 21:39:02.366859: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_1191', 256 bytes spill stores, 256 bytes spill loads



901/901 ━━━━━━━━━━━━━━━━━━━━ 615s 664ms/step - accuracy: 0.3335 - loss: 1.7391 - val_accuracy: 0.4097 - val_loss: 1.5199 - learning_rate: 0.0010
Epoch 2/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 436s 480ms/step - accuracy: 0.3892 - loss: 1.5564 - val_accuracy: 0.4172 - val_loss: 1.4828 - learning_rate: 0.0010
Epoch 3/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 442s 487ms/step - accuracy: 0.4107 - loss: 1.5078 - val_accuracy: 0.4411 - val_loss: 1.4758 - learning_rate: 0.0010
Epoch 4/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 455s 501ms/step - accuracy: 0.4227 - loss: 1.4917 - val_accuracy: 0.4352 - val_loss: 1.4576 - learning_rate: 0.0010
Epoch 5/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 422s 465ms/step - accuracy: 0.4317 - loss: 1.4762 - val_accuracy: 0.4377 - val_loss: 1.4444 - learning_rate: 0.0010
Epoch 6/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 471s 519ms/step - accuracy: 0.4324 - loss: 1.4660 - val_accuracy: 0.4544 - val_loss: 1.4114 - learning_rate: 0.0010
Epoch 7/50
901/901 ━━━━━━━━━━━━━━━━━━━━ 491s 541ms/step - accuracy: 0.4370 

## Plot Training History

In [ ]:
# Plot training & validation accuracy and loss
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(acc))

    plt.figure(figsize=(14, 5))

    # Accuracy plot
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')

    # Loss plot
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')

    plt.show()

plot_history(history)

## Evaluate the Model - Confusion Matrix and Classification Report

In [ ]:
# Confusion matrix and classification report
def plot_confusion_matrix_and_report(model, validation_generator):
    Y_pred = model.predict(validation_generator)
    y_pred = np.argmax(Y_pred, axis=1)

    true_classes = validation_generator.classes
    class_labels = list(validation_generator.class_indices.keys())

    # Create confusion matrix
    cm = confusion_matrix(true_classes, y_pred)

    # Plot the confusion matrix
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
    plt.ylabel('True Labels')
    plt.xlabel('Predicted Labels')
    plt.title('Confusion Matrix')
    plt.show()

    # Print classification report
    print(classification_report(true_classes, y_pred, target_names=class_labels))

plot_confusion_matrix_and_report(final_model, validation_generator)

## Visualize Model Performance on Random Validation Images

In [ ]:
# Randomly select and display 16 validation images with predictions
def show_random_predictions(model, validation_generator):
    # Get a batch of validation images and their corresponding labels
    validation_images, validation_labels = next(validation_generator)
    predictions = model.predict(validation_images)
    predicted_labels = np.argmax(predictions, axis=1)

    class_labels = list(validation_generator.class_indices.keys())

    plt.figure(figsize=(14, 14))
    for i in range(16):
        idx = random.randint(0, validation_images.shape[0] - 1)
        img = validation_images[idx]
        true_label = np.argmax(validation_labels[idx])
        predicted_label = predicted_labels[idx]

        plt.subplot(4, 4, i + 1)
        plt.imshow(img)
        plt.title(f"True: {class_labels[true_label]} Pred: {class_labels[predicted_label]}")
        plt.axis('off')

    plt.show()

# Example call (ensure `final_model` and `validation_generator` are defined)
show_random_predictions(final_model, validation_generator)

## Evaluate the Model on Validation Data

In [ ]:
# After training, evaluate, plot, and show example predictions
evaluation = final_model.evaluate(validation_generator)
print(f"Validation Loss: {evaluation[0]}")
print(f"Validation Accuracy: {evaluation[1]}")

## Save the Retrained Model

In [ ]:
# Save the retrained model
final_model.save('./models/r2_expression_classification_model.keras')
print("Model saved as 'r2_expression_classification_model.keras'")